In [1]:
from neo4j import GraphDatabase
from typing import List
import json

In [2]:
driver = GraphDatabase.driver("bolt://172.31.128.1:7687/",
                              auth=("neo4j", "1234"))
driver.verify_connectivity()


KeyboardInterrupt: 

In [ ]:
def read_words(dir: str, word: str) -> dict:
    with open(dir + f"/synonyms/{word}.json", 'r', encoding='utf-8') as f:
        rawdata: dict = json.load(f)

    with open(dir + f"/weights/{word}.json", 'r', encoding='utf-8') as f:
        weights: dict = json.load(f)

    dbData = {}

    for key, syns in rawdata.items():
        validSyn = []
        validWeight = []
        for syn in syns:
            if f'{key}+{syn}' in weights:
                validSyn.append(syn)
                validWeight.append(weights[f'{key}+{syn}'])
        dbData[key] = [validSyn, validWeight]
    return dbData

def add_words(tx, first: str, sec: List[str], similarity: float):
    for second, sim in zip(sec, similarity):
        tx.run("MERGE (a:Word {word: $first}) "
                "MERGE (b:Word {word: $second})"
                "MERGE (a)-[:SYN {sim: $sim}]->(b)", first=first, second=second, sim=sim)

with driver.session(database="neo4j") as session:
    dbData = read_words('./data/crawlers/hugelabs/', 'surge')
    for key, (synList, synWeight) in dbData.items():
        session.execute_write(add_words, key, synList, synWeight)

driver.close()